# Лабораторная работа №4

In [80]:
import pandas as pd
import json
import codecs
import io
import nltk
import re
import pymorphy2
from fuzzywuzzy import fuzz
from itertools import combinations

from nltk.corpus import stopwords


In [81]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ekaterina.bryanskaya\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ekaterina.bryanskaya\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!


True

In [82]:
SHINGLE = 3
THRESHOLD = 55

In [83]:
stopWords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer(lang='ru')

### Почистить текст

In [84]:
def delete_stop_words(text):
    return [word for word in text if word not in stopWords]

In [85]:
def format_data(text):
    # fmtTextStr = re.sub(r'[^\w\s]', ' ', text.lower(), flags=re.UNICODE)
    fmtTextStr = re.sub(r'[^а-яА-Яa-zA-Z ]', '', text.lower(), flags=re.UNICODE)
    tokenTextArr = nltk.word_tokenize(fmtTextStr)
    cleanTextArr = delete_stop_words(tokenTextArr)
    return " ".join(cleanTextArr)

In [86]:
def get_normalFormStr(line):
    return " ".join([morph.parse(word)[0].normal_form for word in line.split() if morph.parse(word)[0].normal_form not in stopWords])

### Шинглы и прочее

In [87]:
def get_shingles(text, shingle_size=3):
    shingle_set = set()
    words = text.split()
    for i in range(len(words) - shingle_size + 1):
        shingle = ' '.join(words[i:i + shingle_size])
        shingle_set.add(shingle)
        
    return shingle_set    


In [88]:
with io.open('./data/debug.jsonl', encoding='utf-8') as json_file:
    json_lst = list(json_file)
    
data_lst = [json.loads(json_str)['passage']['text'] for json_str in json_lst]
print(data_lst)

['"(1) Издательство Viking Press совместно с компанией TradeMobile выпустят мобильное приложение, посвященное Анне Франк, передает The Daily Telegraph. (2) Программа будет включать в себя фрагменты из дневника Анны, озвученные британской актрисой Хеленой Бонэм Картер. (3) Помимо этого, в приложение войдут фотографии и видеозаписи, документы из архива Фонда Анны Франк, план здания в Амстердаме, где Анна с семьей скрывались от нацистов, и факсимильные копии страниц дневника. (4) Приложение, которое получит название Anne Frank App, выйдет 18 октября. (5) Интерфейс программы будет англоязычным. (6) На каких платформах будет доступно Anne Frank App, не уточняется. Анна Франк родилась в Германии в 1929 году. (7) Когда в стране начались гонения на евреев, Анна с семьей перебрались в Нидерланды. (8) С 1942 года члены семьи Франк и еще несколько человек скрывались от нацистов в потайных комнатах дома в Амстердаме, который занимала компания отца Анны. (9) В 1944 году группу по доносу обнаружили 

In [89]:
df = pd.DataFrame(data_lst, columns=['text'])
df['id'] = list(range(len(df.index)))
df.head(5)

,text,id
0,"""(1) Издательство Viking Press совместно с ком...",0
1,"""(1) Студия Epic Games анонсировала бета-тест ...",1
2,"""(1) Компания Warner Bros. рассматривает Бена ...",2
3,(1) На полках лаборатории Лоуренс-Беркли нашли...,3
4,(1) Усилия Лэнгдона оказываются напрасными: ме...,4


In [90]:
df["formatted"] = df.text.apply(lambda textFTale: get_normalFormStr(format_data(textFTale)))
df.head(5)

,text,id,formatted
0,"""(1) Издательство Viking Press совместно с ком...",0,издательство viking press совместно компания t...
1,"""(1) Студия Epic Games анонсировала бета-тест ...",1,студия epic games анонсировать бетатест шутер ...
2,"""(1) Компания Warner Bros. рассматривает Бена ...",2,компания warner bros рассматривать бен аффлек ...
3,(1) На полках лаборатории Лоуренс-Беркли нашли...,3,полка лаборатория лоуренсберкли наслать забыты...
4,(1) Усилия Лэнгдона оказываются напрасными: ме...,4,усилие лэнгдон оказываться напрасный мешок рас...


In [91]:
# pd.set_option('display.max_colwidth', 50)

df['shingle'] = df.formatted.apply(lambda text: get_shingles(text, SHINGLE))

In [92]:
matr = [[fuzz.token_set_ratio(df.loc[elem1, 'shingle'], df.loc[elem2, 'shingle']) for elem2 in df.index] for elem1 in df.index]

In [93]:
clusters = [{i} for i in df.index]
for i in df.index:
    for j in range(i):
        if matr[i][j] < THRESHOLD:
            continue
        for set_i in clusters:
            if i in set_i:
                break
        for set_j in clusters:
            if j in set_j:
                break
        if set_i == set_j:
            continue
        set_ij = set_i.union(set_j)
        clusters.remove(set_i)
        clusters.remove(set_j)
        clusters.append(set_ij)

for cluster in clusters:
    if len(cluster) > 1:
        print("cluster")
        for elem in cluster:
            print(df.loc[elem, 'text'])
        print("----------------------")

cluster
"(1) 5 декабря на акцию оппозиции к памятнику Грибоедову на Чистых прудах, приуроченную к годовщине начала протестов, пришли несколько сотен человек. (2) Корреспондент "Ленты.ру" оценил количество присутствовавших, включая представителей СМИ, в 200 человек. (3) Пользователи твиттера также сообщили о том, что на Чистых прудах собрались от 200 до 300 человек. (4) Между тем РИА Новости сообщает о нескольких задержанных в ходе акции. (5) Портал ovdinfo.org приводит имена двоих из них: это некий Геннадий Строганов и журналистка Наталья Телегина. (6) Отмечается, что Строганов не выкрикивал лозунгов, а в руках у него не было каких-либо плакатов. (7) Среди пришедших к памятнику Грибоедову был сопредседатель ПАРНАСа Борис Немцов, а также журналистка Ольга Романова и инвестор телеканала "Дождь" Александр Винокуров. (8) Все они ранее вошли в Координационный совет оппозиции. (9) 5 декабря 2011 года, на следующий день после проведения выборов в Госдуму, на Чистопрудном бульваре состоялась п